<a href="https://colab.research.google.com/github/utkarsh235/Machine-Learning/blob/master/Extent%20of%20Damage%20Prediction%20by%20Satellite%20Images/satellite-disaster-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import time
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential, Model
from keras.applications import VGG16, VGG19, Xception, ResNet50, ResNet101, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet121, DenseNet169
from keras.layers import Input, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [ ]:
main_folder = '../input/xview2-damage-assessment'

In [ ]:
train_images = train_folder = main_folder + '/train/images'
holdout_folder = main_folder + '/hold'
tier3_folder = main_folder + '/tier3'
tier3_images = tier3_folder + '/images'
holdout_images = holdout_folder+'/images'

train_labels_path = main_folder + '/train/labels'
hold_labels_path = main_folder + '/hold/labels'
tier3_labels_path = main_folder + '/tier3/labels'
#train_labels_path = '../input/xview2-damage-assessment/train/labels'
#hold_labels_path = '../input/xview2-damage-assessment/hold/labels'
#tier3_labels_path = '../input/xview2-damage-assessment/tier3/labels'

#train_images = train_folder = '../input/xview2-damage-assessment/train/images'
#holdout_folder = '../input/xview2-damage-assessment/hold'
#tier3_folder = '../input/xview2-damage-assessment/tier3'
#tier3_images = tier3_folder + '/images'
print(os.listdir(holdout_folder))
print(os.listdir(tier3_folder))

In [ ]:
info = pd.DataFrame(
    {'Name': '_',
     'index': '0',
     'pre_path': '',
     'post_path': '',
     'paths': 'paths'
    },index=range(len(os.listdir(train_folder)) + len(os.listdir(holdout_folder+'/images'))+ len(os.listdir(tier3_images)))
)

In [ ]:
info

In [ ]:
print(len(os.listdir(train_images)))
print(len(os.listdir(holdout_images)))
print(len(os.listdir(tier3_images)))

In [ ]:
start = time.time()
for i in range(len(os.listdir(train_folder))+ len(os.listdir(holdout_images))+ len(os.listdir(tier3_images))):
    if i % 1000 == 0:
        print(i)
    if i < len(os.listdir(train_folder)):
        path = train_folder + '/' + os.listdir(train_folder)[i]
        broken = os.listdir(train_folder)[i].split('_')

        pre_path = train_folder + '/' + broken[0] + '_' + broken[1] + '_' + 'pre_disaster.png'  
        post_path = train_folder + '/' + broken[0] + '_' + broken[1] + '_' + 'post_disaster.png'
    elif i > len(os.listdir(train_folder)) and i < (len(info) - len(os.listdir(tier3_images))):
        path = holdout_images + '/' + os.listdir(holdout_images)[i-len(os.listdir(train_folder))]
        broken = os.listdir(holdout_images)[i-len(os.listdir(train_folder))].split('_')

        pre_path = holdout_images + '/' + broken[0] + '_' + broken[1] + '_' + 'pre_disaster.png'  
        post_path = holdout_images + '/' + broken[0] + '_' + broken[1] + '_' + 'post_disaster.png'
    else:
        path = tier3_images + '/' + os.listdir(tier3_images)[i-len(os.listdir(train_folder))-len(os.listdir(holdout_images))]
        broken = os.listdir(tier3_images)[i-len(os.listdir(train_folder))-len(os.listdir(holdout_images))].split('_')

        pre_path = tier3_images + '/' + broken[0] + '_' + broken[1] + '_' + 'pre_disaster.png'  
        post_path = tier3_images + '/' + broken[0] + '_' + broken[1] + '_' + 'post_disaster.png'
        
        
    info['Name'][i] = broken[0] + '_' + broken[1]
    info['index'][i] = broken[1]
    info['pre_path'][i] = pre_path
    info['post_path'][i] = post_path
    info['paths'][i] = [pre_path, post_path]
end = time.time()
print(f'Operation executed in {end-start} seconds')

In [ ]:
info

In [ ]:
info.drop_duplicates(subset='pre_path', keep='first', inplace=True)
info.reset_index(inplace=True)
info.drop(['level_0'], axis=1, inplace=True)

In [ ]:
info

In [ ]:
print(info['pre_path'][0])
print(info['post_path'][0])
print(info['paths'][0])

In [ ]:
start = time.time()
info['labels'] = ''
info['disaster_type'] = ''
for i in range(len(info)):
    if i%500 == 0:
        print(i)
    
    if i < len(os.listdir(train_folder))//2:
        pass
        json_file = train_labels_path + '/' + info['Name'][i] + '_' + 'post_disaster.json'
        jso_post = pd.read_json(json_file)
        info['disaster_type'][i] =  jso_post['metadata']['disaster_type']
        length = len(jso_post.iloc[1,:][0])
        #print(jso_post.iloc[1, :][0][0]['properties']['subtype'])
        for j in range(length):
            info['labels'][i] = info['labels'][i] + jso_post.iloc[1,:][0][j]['properties']['subtype'] + '_'
    
    elif i > len(os.listdir(train_folder))//2 and i <= ( len(os.listdir(train_folder)) + len(os.listdir(holdout_images)) )//2:
        json_file = hold_labels_path + '/' + info['Name'][i] + '_' + 'post_disaster.json'
        #print(json_file)
        jso_post = pd.read_json(json_file)
        info['disaster_type'][i] =  jso_post['metadata']['disaster_type']
        length = len(jso_post.iloc[1,:][0])
        #print(jso_post.iloc[1, :][0][0]['properties']['subtype'])
        for j in range(length):
            info['labels'][i] = info['labels'][i] + jso_post.iloc[1,:][0][j]['properties']['subtype'] + '_'
    
    else:
        json_file = tier3_labels_path + '/' + info['Name'][i] + '_' + 'post_disaster.json'
        jso_post = pd.read_json(json_file)
        info['disaster_type'][i] =  jso_post['metadata']['disaster_type']
        length = len(jso_post.iloc[1,:][0])
        #print(jso_post.iloc[1, :][0][0]['properties']['subtype'])
        for j in range(length):
            info['labels'][i] = info['labels'][i] + jso_post.iloc[1,:][0][j]['properties']['subtype'] + '_'
        
end = time.time()
print(end-start)

In [ ]:
info

In [ ]:
info['simplified_labels'] = 0
for i in range(len(info)):
    info['simplified_labels'][i] = info['labels'][i].split('_')
    info['simplified_labels'][i] = info['simplified_labels'][i][:-1]
    
for i in range(len(info)):
    if info['simplified_labels'][i] == []:
        info['simplified_labels'][i] = ['no-damage']

In [ ]:
info

In [ ]:
info

In [ ]:
info['simplified_labels'].dtype

In [ ]:
count = 0
for j in range(len(info)):
    if info['simplified_labels'][j] == list(['un-classified']):
        count+=1
count

In [ ]:
count = 0
for j in range(len(info)):
    if info['simplified_labels'][j] == list(["no-damage"]):
        count += 1
print(count)

In [ ]:
for i in range(10):
    pre = info['paths'][i][0]
    post = info['paths'][i][1]
    pre = cv2.imread(pre)
    post = cv2.imread(post)
    print(pre.shape, post.shape)

Every pre and post image is of shape (1024 x 1024 x 3)

In [ ]:
len(np.where(info['labels'] == '')[0])

In [ ]:
info['disaster_type'].value_counts()

In [ ]:
info['regression_labels'] = 1.33

def counter(lst):
    a = Counter(lst)
    num_no_damage = a['no-damage']
    num_minor_damage = a['minor-damage']
    num_major_damage = a['major-damage']
    num_destroyed = a['destroyed']
    num_unclassified = a['un-classified']
    return num_no_damage, num_minor_damage, num_major_damage, num_destroyed, num_unclassified

for i in range(len(info)):
    num_no_damage, num_minor_damage, num_major_damage, num_destroyed, num_unclassified = counter(info['simplified_labels'][i])
    reg_value = ( (num_no_damage*0)+(num_minor_damage*0.33)+(num_major_damage*0.66)+(num_destroyed*1)+(num_unclassified*0.50) ) / len(info['simplified_labels'][i]) 
    info['regression_labels'][i] = reg_value       
            

In [ ]:

info['regression_labels'].value_counts()

In [ ]:
info['regression_labels_str'] = str(info['regression_labels'])
for i in range(len(info)):
    info['regression_labels_str'][i] = str(info['regression_labels'][i])

In [ ]:
for i in range(len(info)):
    info['regression_labels'][i] = float(info['regression_labels_str'][i][0:4])

In [ ]:
info_copied = info.copy()

In [ ]:
info_sampled = info_copied.sample(10).reset_index()

In [ ]:
info_sampled

In [ ]:
X = info_copied.drop(['regression_labels'], axis=1)
Y = info_copied['regression_labels']
train_x, val_x, train_y, val_y = train_test_split(X, Y, shuffle=True, random_state=42)

In [ ]:
batch_size=40
#sampled_info = info.sample(batch_size).reset_index()
def train_generator(sampled_info, batch_size=batch_size):
    while True:
        batch_train_images = []
        batch_train_labels = []
        sampled_dataset = sampled_info.sample(batch_size).reset_index()
        for i in range(len(sampled_dataset)):
            pre_img_path = sampled_dataset['pre_path'][i]
            post_img_path = sampled_dataset['post_path'][i]
            preprocessed_array = read_and_process_img(pre_img_path, post_img_path)
            batch_train_images.append(preprocessed_array)
            batch_train_labels.append(sampled_dataset['regression_labels'][i])
            #print(i)
        
        batch_train_images = np.array(batch_train_images)
        batch_train_labels = np.array(batch_train_labels)
        yield(batch_train_images, batch_train_labels)

def read_and_process_img(pre_img_path, post_img_path):
    img_size=224
    pre_img = cv2.imread(pre_img_path)
    post_img = cv2.imread(post_img_path)
    pre_img = cv2.resize(pre_img, (img_size, img_size))
    post_img = cv2.resize(post_img, (img_size, img_size))
    final_array = np.concatenate((pre_img, post_img), axis=2)
    final_array = final_array / 255
    return final_array

In [ ]:
train_x = pd.concat([train_x, train_y], axis=1)
val_x = pd.concat([val_x, val_y], axis=1)

In [ ]:
#trained = train_generator(info)
for_train = train_generator(train_x)
for_val = train_generator(val_x)

In [ ]:
train_x

In [ ]:
baseModel = tf.keras.applications.Xception(
    include_top=False,
    weights = None,
    #weights="imagenet",
    input_tensor=Input(shape=(224, 224, 6)),
    input_shape=None,
    pooling=None,
    classes=1
)
#baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(20, activation="sigmoid")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel) # Try using sigmoid as well

for layer in baseModel.layers:
    layer.trainable = False
    
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'mean_squared_error',
    metrics = ['accuracy', 'mse']
)

In [ ]:
history = model.fit(for_train, steps_per_epoch=len(train_x)//batch_size, epochs=5, validation_data=for_val, validation_steps=len(val_x)//batch_size)

In [ ]:
output_result_path = './'
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = output_result_path + 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
# Insert Output Path Below
output_path = './'
tf.keras.models.save_model(model, output_path+'model_xception.hdf5')

In [ ]:
img_size = 224
some_images = []
some_labels = []
for i in range(200):
    pre_img = info['pre_path'][i]
    post_img = info['post_path'][i]
    pre = cv2.imread(pre_img)
    post = cv2.imread(post_img)
    pre = cv2.resize(pre, (img_size,img_size))
    post = cv2.resize(post, (img_size,img_size))
    final = np.concatenate((post, pre), axis=2)
    final = final/255
    some_images.append(final)
    some_labels.append(info['regression_labels'][i])
some_images = np.array(some_images)

In [ ]:
pred = model.predict(some_images)

In [ ]:
pred

In [ ]:
some_labels

In [ ]:
info_copied['regression_labels'].value_counts()

In [ ]:
model.evaluate(some_images, np.array(some_labels))